# PART 5 - SQLITE

## TOPIC 1 - Complete Overview - Creating a Database, Table, and Running Queries

- create a connection named conn conn = sqlite3.connect(':memory:') #using memory in RAM
- OR create a connection named `conn` in current working directory by connecting to employee.db
- if the database doesn't exist --> create new database employee.db 
- if the database does exist --> connect to the database employee.db

In [1]:
import sqlite3
conn = sqlite3.connect('employee.db')
c = conn.cursor()
c.execute("DROP TABLE employees")
conn.commit()
conn.close()

OperationalError: no such table: employees

4 steps to run SQL query using python sqlite3
- Create connection with conn = `sqlite3.connect(<...db>)`
- Create cursor for a connection with c = conn`.cursor()`
- Execute query from connection with c`.execute("""<query>""")` . Note: triple quotes to add string in multiple rows
- Commit query conn`.commit()`
- Close connection after running query conn`.close()`

In [2]:
import sqlite3
conn = sqlite3.connect('employee.db')
#create a cursor
c = conn.cursor()
c.execute("""CREATE TABLE employees (
                first text,
                last text,
                pay integer)""")
conn.commit()
conn.close()

For more data format when creating a new table using SQLite http://www.sqlite.org/datatype3.html

In [3]:
import sqlite3
conn = sqlite3.connect('employee.db')
c = conn.cursor()
c.execute("INSERT INTO employees VALUES ('First1','Last1',70000)")
conn.commit()
conn.close()

Multiple ways to view query result:
- `fetchone()`: show only 1 row of result    
- `fetchmany(n)`: show n rows of result
- `fetchall()`: show all result

In [4]:
import sqlite3
conn = sqlite3.connect('employee.db')
c = conn.cursor()
c.execute("SELECT * FROM employees WHERE last = 'Last1'")
print(c.fetchone())
conn.commit()
conn.close()

('First1', 'Last1', 70000)


In [5]:
import sqlite3
conn = sqlite3.connect('employee.db')
c = conn.cursor()
c.execute("INSERT INTO employees VALUES ('First2','Last1',50000)")
conn.commit()
c.execute("SELECT * FROM employees WHERE last = 'Last1'")
print(c.fetchall())
conn.commit()
conn.close()

[('First1', 'Last1', 70000), ('First2', 'Last1', 50000)]


In [6]:
#%run employee.ipynb
class Employee:
    raise_amount = 1.04
    def __init__(self,first,last,pay):
        self.first = first
        self.last = last
        self.pay = pay
        self.email = first + '.' + last + '@test.com'
    def fullname(self): #must add self here
        return '{} {}'.format(self.first, self.last)
    def apply_raise(self):
        self.pay = int(self.pay*self.raise_amount)

In [7]:
conn = sqlite3.connect('employee.db')
c = conn.cursor()
emp_1=Employee('First3','Last2',80000)
emp_2=Employee('First4','Last2',90000)
c.execute("INSERT INTO employees VALUES (?,?,?)",(emp_1.first,emp_1.last,emp_1.pay))
conn.commit()
c.execute("INSERT INTO employees VALUES (:first,:last,:pay)",
          {'first':emp_2.first,'last':emp_2.last,'pay':emp_2.pay})
conn.commit()
c.execute("SELECT * FROM employees WHERE last = ?",('Last1',))
print(c.fetchall())
c.execute("SELECT * FROM employees WHERE last =:last",{'last':'Last2'})
print(c.fetchall())
conn.close()

[('First1', 'Last1', 70000), ('First2', 'Last1', 50000)]
[('First3', 'Last2', 80000), ('First4', 'Last2', 90000)]


Run create table and all insert statements using `:memory:` to run the code in RAM. This can be __run multiple times__ without any error like the table already exists

In [9]:
#%run employee.ipynb
import sqlite3
conn = sqlite3.connect(':memory:')
c = conn.cursor()
c.execute("""CREATE TABLE employees (
                first text,
                last text,
                pay integer
            )""")
conn.commit()
c.execute("INSERT INTO employees VALUES ('First1','Last1',70000)")
c.execute("INSERT INTO employees VALUES ('First2','Last1',50000)")
conn.commit()
emp_1=Employee('First3','Last2',80000)
emp_2=Employee('First4','Last2',90000)
c.execute("INSERT INTO employees VALUES (?,?,?)",(emp_1.first,emp_1.last,emp_1.pay))
conn.commit()
c.execute("INSERT INTO employees VALUES (:first,:last,:pay)",
          {'first':emp_2.first,'last':emp_2.last,'pay':emp_2.pay})
conn.commit()
c.execute("SELECT * FROM employees WHERE last = ?",('Last1',))
print(c.fetchall())
c.execute("SELECT * FROM employees WHERE last =:last",{'last':'Last2'})
print(c.fetchall())
conn.commit()
conn.close()

[('First1', 'Last1', 70000), ('First2', 'Last1', 50000)]
[('First3', 'Last2', 80000), ('First4', 'Last2', 90000)]


There are 2 ways to use placeholders for SQLite query:
- Quesion mark syntax: Use `?` as placeholder in the query string then pass a tuple of values in round brackets after comma
- Dictionary syntax: Use colon `:` before placeholder(s)' name then pass a dictionary for corresponding values of each placholder

__Note:__ When there is only one placeholder, the passed value need to be followed by 1 comma `,` for the question mark syntax

The 2 examples below use exactly the same query but with different placeholder syntax

In [11]:
#%run employee.ipynb
conn = sqlite3.connect(':memory:')
c = conn.cursor()
c.execute("""CREATE TABLE employees (
                first text,
                last text,
                pay integer
            )""")
def insert_emp(emp):
    with conn:
        c.execute("INSERT INTO employees VALUES (:first,:last,:pay)",
                  {'first':emp.first,'last':emp.last,'pay':emp.pay})
def get_emps_by_name(lastname):
    c.execute("SELECT * FROM employees WHERE last =:last",{'last':lastname})
    return c.fetchall()
def update_pay(emp,pay):
    with conn:
        c.execute("""UPDATE employees SET pay = :pay
                     WHERE first = :first AND last = :last""",
                  {'first':emp.first,'last':emp.last, 'pay':pay})
def remove_emp(emp):
    with conn:
        c.execute("DELETE FROM employees WHERE first = :first AND last = :last",
                  {'first':emp.first,'last':emp.last})

emp_1=Employee('First1','Last',80000)
emp_2=Employee('First2','Last',90000)
insert_emp(emp_1)
insert_emp(emp_2)
emps = get_emps_by_name('Last')
print(emps)
update_pay(emp_2,95000)
remove_emp(emp_1)
emps = get_emps_by_name('Last')
print(emps)
conn.close()


[('First1', 'Last', 80000), ('First2', 'Last', 90000)]
[('First2', 'Last', 95000)]


In [14]:
#%run employee.ipynb
import sqlite3
conn = sqlite3.connect(':memory:')
c = conn.cursor()
c.execute("""CREATE TABLE employees (
                first text,
                last text,
                pay integer
                )""")
def insert_emp(emp):
    with conn:
        c.execute("INSERT INTO employees VALUES (?,?,?)",(emp.first, emp.last,emp.pay))
def get_emps_by_name(lastname):
    c.execute("SELECT * FROM employees WHERE last =?",(lastname,))
    return c.fetchall()
def update_pay(emp,pay):
    with conn:
        c.execute("""UPDATE employees SET pay = ? WHERE first = ? AND last = ?""",
                  (pay,emp.first,emp.last))
def remove_emp(emp):
    with conn:
        c.execute("DELETE FROM employees WHERE first = ? AND last = ?",(emp.first,emp.last))
emp_1=Employee('First1','Last',80000)
emp_2=Employee('First2','Last',90000)
insert_emp(emp_1)
insert_emp(emp_2)
emps = get_emps_by_name('Last')
print(emps)
update_pay(emp_2,95000)
remove_emp(emp_1)
emps = get_emps_by_name('Last')
print(emps)
conn.close()

[('First1', 'Last', 80000), ('First2', 'Last', 90000)]
[('First2', 'Last', 95000)]
